# CUPS Report Analisis Function

In [ ]:
import pandas as pd
import numpy as np
from datetime import timedelta

In [2]:
df = pd.read_excel('/home/raul/CAREER/DATA_SCIENCE/proyectos/CUPX/cups-examples/InformeCUPS_AMOROS.xlsx', sheet_name=1)
df.drop('Tipo lectura', axis=1, inplace=True)
df.head(40)

,Fecha lectura anterior,Fecha lectura,Consumo P1,Consumo P2,Consumo P3,Consumo P4,Consumo P5,Consumo P6
0,30/11/2020,01/12/2020,0,1122,771,0,0,1455
1,01/12/2020,01/01/2021,33773,23115,0,0,0,50226
2,01/01/2021,01/02/2021,36772,24989,0,0,0,44394
3,01/02/2021,01/03/2021,32760,22241,0,0,0,40610
4,01/03/2021,01/04/2021,0,33989,23429,0,0,50449
5,01/04/2021,01/05/2021,0,0,0,36068,25461,49314
6,01/05/2021,31/05/2021,0,0,0,33233,24018,47816
7,31/05/2021,30/06/2021,0,0,30461,22779,0,37340
8,30/06/2021,31/07/2021,32269,23738,0,0,0,44815
9,31/07/2021,31/08/2021,0,0,37129,26828,0,52685


In [3]:
def dates_fix(df):
    df_copy = df.copy()

    # Delete blank spaces
    df_copy['Fecha lectura anterior'] = df_copy['Fecha lectura anterior'].str.strip()
    df_copy['Fecha lectura'] = df_copy['Fecha lectura'].str.strip()

    # Convert date columns to pandas datetime
    df_copy['Fecha lectura anterior'] = pd.to_datetime(df_copy['Fecha lectura anterior'], format='%d/%m/%Y')
    df_copy['Fecha lectura'] = pd.to_datetime(df_copy['Fecha lectura'], format='%d/%m/%Y')

    # Calculating interval length
    df_copy['Duración días'] = (df_copy['Fecha lectura'] - df_copy['Fecha lectura anterior']).dt.days

    
    return df_copy

#### Fx to Redistribute 1 day intervals

In [4]:
def distrib_1day_rows(df):
    for i, row in df.iterrows():        

        # Check if row interval is just 1 day
        if row['Duración días'] == 1:
            # We extract the date's month
            actual_month = row['Fecha lectura'].month

            # Check previous row to si if they have the same month
            if i > 0 and df.loc[i-1, 'Fecha lectura'].month == actual_month:
                # Has the same month as previous row? --> sum the consumption for every period
                for p in range(1, 7):  # P1 a P6
                    column = f'Consumo P{p}'
                    if column in df.columns:
                        df.loc[i-1, column] += row.get(column, 0)
            # Previous row hasn't the same month, check next row
            elif i < len(df) - 1 and df.loc[i+1, 'Fecha lectura'].month == actual_month:
                # Has the same month as next row? --> sum the consumption for every period
                for p in range(1, 7):  # P1 a P6
                    column = f'Consumo P{p}'
                    if column in df.columns:
                        df.loc[i+1, column] += row.get(column, 0)
    
    # Delete 1 day rows after distributing consumption
    no_1dayrows_df = df[df['Duración días'] != 1].copy().reset_index(drop=True)
    return no_1dayrows_df



### Defining Periods by Month


In [5]:
tabla_periodos = pd.read_excel('/home/raul/CAREER/DATA_SCIENCE/proyectos/CUPX/cups-examples/TABLA PERIODOS.xlsx')
periodict = {i+1:list(pd.unique(tabla_periodos[m])[::-1]) for i, m in enumerate(tabla_periodos.columns)}
periodict

{1: ['P1', 'P2', 'P6'],
 2: ['P1', 'P2', 'P6'],
 3: ['P2', 'P3', 'P6'],
 4: ['P4', 'P5', 'P6'],
 5: ['P4', 'P5', 'P6'],
 6: ['P3', 'P4', 'P6'],
 7: ['P1', 'P2', 'P6'],
 8: ['P3', 'P4', 'P6'],
 9: ['P3', 'P4', 'P6'],
 10: ['P4', 'P5', 'P6'],
 11: ['P2', 'P3', 'P6'],
 12: ['P1', 'P2', 'P6']}

In [6]:
periodict = {1: ['P1', 'P2', 'P6'],
 2: ['P1', 'P2', 'P6'],
 3: ['P2', 'P3', 'P6'],
 4: ['P4', 'P5', 'P6'],
 5: ['P4', 'P5', 'P6'],
 6: ['P3', 'P4', 'P6'],
 7: ['P1', 'P2', 'P6'],
 8: ['P3', 'P4', 'P6'],
 9: ['P3', 'P4', 'P6'],
 10: ['P4', 'P5', 'P6'],
 11: ['P2', 'P3', 'P6'],
 12: ['P1', 'P2', 'P6']}

In [33]:
def div_rows(df):
    splited_rows = []
    work_df = dates_fix(df)
    
    # Loop through each row in the original DataFrame
    for _, row in work_df.iterrows():
        start_date = row['Fecha lectura anterior']
        end_date = row['Fecha lectura']
        consumption_per_period = {f'Consumo P{i}': row[f'Consumo P{i}'] for i in range(1, 7)}  # Get consumption for P1 to P6
        total_days = row['Duración días']

        # If dates span over different months, split the row
        if start_date.month != end_date.month or start_date.year != end_date.year:
            # Get periods for each month
            periods_part1 = periodict[start_date.month]  # First month periods
            periods_part2 = periodict[end_date.month]  # Second month periods
            
            # Find the shared periods between both months
            shared_periods = set(periods_part1).intersection(periods_part2)
            
            # Get the periods that are unique to each month
            unique_periods_part1 = set(periods_part1) - shared_periods
            unique_periods_part2 = set(periods_part2) - shared_periods

            # First part: from start_date to the last day of start month
            last_day_of_start_month = pd.Timestamp(start_date.year, start_date.month, 
                                                   pd.Timestamp(start_date.year, start_date.month, 1).days_in_month)
            days_part1 = (last_day_of_start_month - start_date).days + 1
            prop_days1 = days_part1 / total_days

            # Distribute consumption proportionally for shared periods
            consumptions_part1 = {
                f'Consumo P{p[-1]}': round(prop_days1 * consumption_per_period[f'Consumo P{p[-1]}'], 0)
                for p in shared_periods
            }
            # Assign full consumption for unique periods in the first month
            consumptions_part1.update({
                f'Consumo P{p[-1]}': consumption_per_period[f'Consumo P{p[-1]}']
                for p in unique_periods_part1
            })

            splited_rows.append({
                'Fecha lectura anterior': start_date, 
                'Fecha lectura': last_day_of_start_month,
                'Duración días': days_part1,
                'Proporción días': prop_days1,
                **consumptions_part1
            })

            # Second part: from the first day of the next month to end_date
            first_day_of_next_month = last_day_of_start_month + timedelta(days=1)
            days_part2 = (end_date - first_day_of_next_month).days + 1
            prop_days2 = days_part2 / total_days

            # Distribute consumption proportionally for shared periods in the second month
            consumptions_part2 = {
                f'Consumo P{p[-1]}': consumption_per_period[f'Consumo P{p[-1]}'] - consumptions_part1.get(f'Consumo P{p[-1]}', 0)
                for p in shared_periods
            }
            # Assign full consumption for unique periods in the second month
            consumptions_part2.update({
                f'Consumo P{p[-1]}': consumption_per_period[f'Consumo P{p[-1]}']
                for p in unique_periods_part2
            })

            splited_rows.append({
                'Fecha lectura anterior': first_day_of_next_month, 
                'Fecha lectura': end_date,
                'Duración días': days_part2,
                'Proporción días': prop_days2,
                **consumptions_part2
            })

        else:
            # If the dates are within the same month, keep the row as is with all its consumption
            month_periods = periodict[start_date.month]
            month_consumptions = {
                f'Consumo P{p[-1]}': consumption_per_period[f'Consumo P{p[-1]}'] for p in month_periods
            }
            
            splited_rows.append({
                'Fecha lectura anterior': start_date, 
                'Fecha lectura': end_date,
                'Duración días': total_days,
                'Proporción días': 1,
                **month_consumptions
            })

    # Create DataFrame with the split rows and redistribute any 1-day rows
    pre_df = pd.DataFrame(splited_rows)
    pre_df['Mes'] = pre_df['Fecha lectura anterior'].dt.strftime('%B')
    final_df = distrib_1day_rows(pre_df)

    # Reorder the columns to keep things consistent
    orden_columnas = ['Fecha lectura anterior', 'Fecha lectura', 'Duración días', 'Proporción días', 
                  'Consumo P1', 'Consumo P2', 'Consumo P3', 'Consumo P4', 'Consumo P5', 'Consumo P6', 'Mes']
    final_df = final_df[orden_columnas].fillna(0)

    return final_df

# Apply the function to split rows and display the result
df_dividido = div_rows(df)

# Show the resulting DataFrame
df_dividido.head(5)


,Fecha lectura anterior,Fecha lectura,Duración días,Proporción días,Consumo P1,Consumo P2,Consumo P3,Consumo P4,Consumo P5,Consumo P6,Mes
0,2020-12-01,2020-12-31,31,1.0,33773.0,23115.0,0.0,0.0,0.0,50226.0,December
1,2021-01-01,2021-01-31,31,1.0,36772.0,24989.0,0.0,0.0,0.0,44394.0,January
2,2021-02-01,2021-02-28,28,1.0,32760.0,22241.0,0.0,0.0,0.0,40610.0,February
3,2021-03-01,2021-03-31,31,1.0,0.0,33989.0,23429.0,0.0,0.0,50449.0,March
4,2021-04-01,2021-04-30,30,1.0,0.0,0.0,0.0,36068.0,25461.0,49314.0,April


In [34]:
df_dividido.columns

Index(['Fecha lectura anterior', 'Fecha lectura', 'Duración días',
       'Proporción días', 'Consumo P1', 'Consumo P2', 'Consumo P3',
       'Consumo P4', 'Consumo P5', 'Consumo P6', 'Mes'],
      dtype='object')

In [24]:
columns_names = [f'Consumo P{i}' for i in range(1,7)]
totals_mod = {columns_names[i]:df_dividido.loc[:,p].sum() for i, p in enumerate(columns_names)}
totals_mod

{'Consumo P1': np.float64(327065.0),
 'Consumo P2': np.float64(372337.0),
 'Consumo P3': np.float64(394640.0),
 'Consumo P4': np.float64(471676.0),
 'Consumo P5': np.float64(192998.0),
 'Consumo P6': np.float64(1550876.0)}

In [25]:
totals_og = {columns_names[i]:df.loc[:,p].sum() for i, p in enumerate(columns_names)}
totals_og

{'Consumo P1': np.int64(327065),
 'Consumo P2': np.int64(373459),
 'Consumo P3': np.int64(395411),
 'Consumo P4': np.int64(471676),
 'Consumo P5': np.int64(192998),
 'Consumo P6': np.int64(1552331)}

In [26]:
diff_dict = {f'Diff P{i}':(totals_og[f'Consumo P{i}'] - totals_mod[f'Consumo P{i}']) for i in range(1,7)}
diff_dict

{'Diff P1': np.float64(0.0),
 'Diff P2': np.float64(1122.0),
 'Diff P3': np.float64(771.0),
 'Diff P4': np.float64(0.0),
 'Diff P5': np.float64(0.0),
 'Diff P6': np.float64(1455.0)}

This happens because the first row has to be deleted during the process and we lose those amounts, so the difference between the processed df and the original one is actually 0, meaning full accuracy

In [27]:
totals_og_wo_1row = {columns_names[i]:df.drop(0).loc[:,p].sum() for i, p in enumerate(columns_names)}
diff_dict_wo_1row = {f'Diff P{i}':(totals_og_wo_1row[f'Consumo P{i}'] - totals_mod[f'Consumo P{i}']) for i in range(1,7)}
diff_dict_wo_1row

{'Diff P1': np.float64(0.0),
 'Diff P2': np.float64(0.0),
 'Diff P3': np.float64(0.0),
 'Diff P4': np.float64(0.0),
 'Diff P5': np.float64(0.0),
 'Diff P6': np.float64(0.0)}

## : )

In [40]:
def join_per_month(df):
    work_df = div_rows(df.copy())  # Make a copy of the DataFrame to avoid modifying the original
    i = 0  # Initialize index to loop through the DataFrame

    while i < len(work_df) - 1:  # Loop through until the second-to-last row
        # Get the current month and the next row's month
        actual_month = work_df.loc[i, 'Mes']
        next_row_month = work_df.loc[i + 1, 'Mes']

        # If the current row's month matches the next row's month
        if actual_month == next_row_month:
            # Update the end date to match the next row's end date
            work_df.loc[i, 'Fecha lectura'] = work_df.loc[i + 1, 'Fecha lectura']

            # Add up the consumption values for P1 to P6
            for p in range(1, 7):
                if f'Consumo P{p}' in work_df.columns:
                    work_df.loc[i, f'Consumo P{p}'] += work_df.loc[i + 1, f'Consumo P{p}']

            # Drop the next row since we've merged it
            work_df = work_df.drop(i + 1).reset_index(drop=True)
        else:
            i += 1  # If they're different months, just move to the next row

    return work_df


In [41]:
df_unido = join_per_month(df)
df_unido.head(40)

,Fecha lectura anterior,Fecha lectura,Duración días,Proporción días,Consumo P1,Consumo P2,Consumo P3,Consumo P4,Consumo P5,Consumo P6,Mes
0,2020-12-01,2020-12-31,31,1.000000,33773.0,23115.0,0.0,0.0,0.0,50226.0,December
1,2021-01-01,2021-01-31,31,1.000000,36772.0,24989.0,0.0,0.0,0.0,44394.0,January
2,2021-02-01,2021-02-28,28,1.000000,32760.0,22241.0,0.0,0.0,0.0,40610.0,February
3,2021-03-01,2021-03-31,31,1.000000,0.0,33989.0,23429.0,0.0,0.0,50449.0,March
4,2021-04-01,2021-04-30,30,1.000000,0.0,0.0,0.0,36068.0,25461.0,49314.0,April
5,2021-05-01,2021-05-31,30,1.000000,0.0,0.0,0.0,33992.0,24018.0,49061.0,May
6,2021-06-01,2021-06-30,30,1.000000,0.0,0.0,30461.0,22020.0,0.0,37541.0,June
7,2021-07-01,2021-07-31,31,1.000000,32269.0,23738.0,0.0,0.0,0.0,45069.0,July
8,2021-08-01,2021-08-31,31,1.000000,0.0,0.0,38308.0,27690.0,0.0,52527.0,August
9,2021-09-01,2021-09-30,30,1.000000,0.0,0.0,34193.0,25982.0,0.0,46315.0,September


## CLASS DEFINITION

---

In [11]:
class CupsARP:

    def __init__(self, path):
        self.path = path
        self.page_list = []

        self.df_cups_list = pd.read_excel(path, sheet_name=0)
        self.df_activa = pd.read_excel(path, sheet_name=1).drop('Tipo lectura', axis=1)
        self.df_maxim = pd.read_excel(path, sheet_name=2)
        self.df_react = pd.read_excel(path, sheet_name=3)

        self.page_list.extend([self.df_cups_list, self.df_activa, self.df_maxim, self.df_react])

    
    def load_page(self, param):
        return self.page_list[param]


    def ly_consum(self):
        self.consums = {}
        
        self.tot_cons = self.df_activa[['Fecha lectura','Consumo P1', 'Consumo P2',
       'Consumo P3', 'Consumo P4', 'Consumo P5', 'Consumo P6']].sum().sum()
        self.consums['TOTAL'] = self.tot_cons

        for i in range(1,7):
             name = f'P{i}'
             
             cons = self.df_activa.loc[:,f'Consumo P{i}'].sum()
             self.consums[name] = cons

        return self.consums
    
    def year_table(self):
        # Crear una tabla pivotada con los meses y periodos P1 a P6
        tabla = self.df_activa[['Fecha lectura', 'Consumo P1', 'Consumo P2',
                                'Consumo P3', 'Consumo P4', 'Consumo P5', 'Consumo P6']].copy()

        # Convertir las fechas a meses (ej. Enero, Febrero, ...)
        tabla['Mes'] = pd.to_datetime(self.df_activa['Fecha lectura'], format="%d/%m/%Y").dt.strftime('%B').str.upper()

        # Agrupar los datos por 'Mes' y sumar los consumos para cada mes y cada periodo
        tabla_agrupada = tabla.groupby('Mes').sum()

        # Calcular el total por cada mes
        tabla_agrupada['Total mes'] = tabla_agrupada.sum(axis=1)

        # Calcular el total por cada periodo (columna)
        total_periodos = tabla_agrupada[['Consumo P1', 'Consumo P2', 'Consumo P3',
                                         'Consumo P4', 'Consumo P5', 'Consumo P6']].sum()

        # Añadir una fila de total de periodos al final de la tabla
        tabla_agrupada.loc['Total periodo'] = total_periodos
        tabla_agrupada.loc['Total periodo', 'Total mes'] = total_periodos.sum()

        return tabla_agrupada

